In [1]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit
from autogen.mcp.mcp_proxy import MCPProxy

/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'keep_untouched' has been renamed to 'ignored_types'
  warnings.warn(message, UserWarning)


In [2]:
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [3]:
openapi_json = Path("trello-openapi.json").read_text()
print(openapi_json[:100])

{
  "openapi": "3.0.0",
  "info": {
    "title": "Trello REST API",
    "version": "0.0.1"
  },
  "s


In [4]:
import shutil

tmp_path = Path("mcp_servers") / "servers" / "whatsapp"
shutil.rmtree(tmp_path, ignore_errors=True)
tmp_path.mkdir(parents=True, exist_ok=True)

MCPProxy.create(
    # openapi_url="https://dac-static.atlassian.com/cloud/trello/swagger.v3.json",  
    # openapi_json= openapi_json,
    openapi_url="https://dev.infobip.com/openapi/products/whatsapp.json",
    client_source_path=tmp_path,
    servers=[{"url": "http://127.0.0.1:8000/"}],
)

assert tmp_path.exists(), "Failed to create tmp directory"
# assert list(tmp_path.glob("*.py")) >= 2, "Failed to generate OpenAPI client files"

/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/datamodel_code_generator/parser/jsonschema.py:338: UserWarning: format of 'string' not understood for 'string' - using default
  warn(f'format of {format__!r} not understood for {type_!r} - using default' '')
Unsupported type 'oauth2' and schema_parameters '{'type': 'oauth2', 'description': 'Similarly to the IBSSO Token authentication you can use OAuth 2.0 bearer token with Infobip serving both as resource and authorization server. You can obtain\nthe access token using the client credentials grant from `auth/1/oauth2/token` endpoint. It will provide you with your access token, and its expiration period.\nYou can use the token to authorize your API calls until it expires. You can find out more about the process in\nthe [official specification](https://tools.ietf.org/html/rfc6749#section-4.4).\n\nYou can include your access token in the Authorization HTTP request header like this:\n\n```http\nAuthorization: Be

In [5]:
llm_config = LLMConfig.from_json(path="OAI_CONFIG_LIST").where(model="gpt-4o-mini")
llm_config

LLMConfig(config_list=[{'api_type': 'openai', 'model': 'gpt-4o-mini', 'api_key': '**********', 'tags': ['gpt-4o-mini', 'tool', 'vision']}, {'api_type': 'azure', 'model': 'gpt-4o-mini', 'api_key': '**********', 'api_version': '2024-05-01-preview', 'base_url': 'https://dev-openai-eastus-airt.openai.azure.com/', 'tags': ['gpt-4o-mini', 'tool', 'vision']}])

In [6]:
async def create_toolkit_and_run(session: ClientSession) -> None:
    with llm_config:
        # Create a toolkit with available MCP tools
        toolkit = await create_toolkit(session=session)
        agent = AssistantAgent(name="assistant")
        # Register MCP tools with the agent
        toolkit.register_for_llm(agent)

        # Make a request using the MCP tool
        result = await agent.a_run(
            message="List all Whatsapp tools available by their name and parameters and then categorize them into groups.",
            tools=toolkit.tools,
            max_turns=5,
            user_input=False,
        )

        await result.process()

In [7]:
# Create server parameters for stdio connection

# security = APIKeyHeader.Parameters(value="some_api_key")

server_params = StdioServerParameters(
    # add security parameters here as env var
    command="python",  # The command to run the server
    args=[
        str(tmp_path / "main.py"),  # Path to the server script
        "stdio",
    ],  # Path to server script and transport mode
    # env={
    #     "SECURITY": str(security.dump()),
    # },
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()
    await create_toolkit_and_run(session)

/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'submit_whatsapp_conversions' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'add_whatsapp_sender' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'retry_whatsapp_sender_verification' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: Us

user (to assistant):

List all Whatsapp tools available by their name and parameters and then categorize them into groups.

--------------------------------------------------------------------------------


/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'deprecated_get_whatsapp_templates' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'deprecated_create_whatsapp_template' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable_agent.py:3429: UserWarning: Function 'get_account_whatsapp_templates' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/davor/Projects/ag2/mcp-servers/.venv-3.11/lib/python3.11/site-packages/autogen/agentchat/conversable

assistant (to user):

Here's a comprehensive list of the WhatsApp tools available, categorized into relevant groups:

### 1. **WhatsApp Sender Management**
- **add_whatsapp_sender**
  - Parameters: `business_account_id`, `body`
    - `countryCode`, `phoneNumber`, `displayName`, `type`, `locale`

- **verify_whatsapp_sender**
  - Parameters: `sender`, `body`
    - `code`

- **retry_whatsapp_sender_verification**
  - Parameters: `sender`, `body`
    - `type`, `locale`

- **get_whatsapp_sender_business_info**
  - Parameters: `sender`

- **update_whatsapp_sender_business_info**
  - Parameters: `sender`, `body`
    - `about`, `address`, `description`, `email`, `vertical`, `websites`, `logoUrl`

- **get_whatsapp_senders_quality**
  - Parameters: `senders`

### 2. **WhatsApp Message Sending**
- **send_whatsapp_text_message**
  - Parameters: `body`
    - `from`, `to`, `messageId`, `content`
      - `text`, `previewUrl`

- **send_whatsapp_image_message**
  - Parameters: `body`
    - `from`, `to`